## 교차 검증과 그리드 서치

### 검증 세트

검증 세트(validation set)
- 테스트 세트를 사용하지 않으면 모델이 과대적합인지 과소적합인지 판단하기 어려움

- 테스트 세트를 사용하지 않고 훈련 세트를 또 나누는 방법

- 훈련 세트에서 모델을 훈련하고 검증 세트로 모델을 평가


In [1]:
import pandas as pd
wine = pd.read_csv('https://bit.ly/wine_csv_data')

In [2]:
# class 열을 타깃으로 설정, 나머지 열은 특성 배열에 저장
data = wine[['alcohol', 'sugar', 'pH']].to_numpy()
target = wine['class'].to_numpy()

In [3]:
from sklearn.model_selection import train_test_split
train_input, test_input, train_target, test_target = train_test_split(
    data, target, test_size=0.2, random_state=42)

In [4]:
# 훈련세트와 검증세트 만들기
sub_input, val_input, sub_target, val_target = train_test_split(
    train_input, train_target, test_size=0.2, random_state=42)

In [5]:
# 훈련 세트와 검증 세트의 크기 확인
print(sub_input.shape, val_input.shape)

(4157, 3) (1040, 3)


In [6]:
# 모델 생성, 평가
from sklearn.tree import DecisionTreeClassifier
dt = DecisionTreeClassifier(random_state=42)
dt.fit(sub_input, sub_target)
print(dt.score(sub_input, sub_target))
print(dt.score(val_input, val_target))

0.9971133028626413
0.864423076923077


### 교차 검증

교차 검증(cross validation)
- 검증 세트를 떼어 내어 평가하는 과정을 여러 번 반복
- 이 점수를 평균하여 최종 검증 점수를 얻는다

- 안정적인 검중 점수를 얻고 훈련에 더 많은 데이터 사용 가능

k-폴드 교차 검증(k-fold cross validation)
- 훈련 세트를 몇 부분으로 나누냐에 따라 다르게 부름

3-폴드 교차 검증
- 훈련 세트를 세 부분으로 나눠서 교차 검증을 수행


cross_validate()
- 교차 검증 함수

- 먼저 평가할 모델 객체를 첫 번째 매개변수로 전달
- 훈련 세트 전체를 cross_validate() 함수에 전달

cross_val_score()
- cross_validate() 함수의 결과 중에서 test_score 값만 반환

In [7]:
from sklearn.model_selection import cross_validate
scores = cross_validate(dt, train_input, train_target)
print(scores)

{'fit_time': array([0.00842094, 0.00823712, 0.00778294, 0.00738096, 0.00691605]), 'score_time': array([0.00092793, 0.00067687, 0.00058913, 0.00055695, 0.00049591]), 'test_score': array([0.87019231, 0.84615385, 0.87680462, 0.84889317, 0.83541867])}


- 훈련과 검증 시간은 코랩에서 리소스를 사용하는 상황에 따라 달라질 수 있음

In [8]:
# 교차 검증 최종 점수
import numpy as np
print(np.mean(scores['test_score']))

0.8554925223957948


- 교차 검증을 수행하면 입력한 모델에서 얻을 수 있는 최상의 검증 점수를 가늠해볼 수 있음

- cross_validate()는 훈련 세트를 섞어 폴드를 나누지 않음

- 만약 교차 검증을 할 때 훈련 세트를 섞으려면 분할기(spliter)를 지정해야 함

In [9]:
# 교차 검증
from sklearn.model_selection import StratifiedKFold
scores = cross_validate(dt, train_input, train_target,
                        cv=StratifiedKFold())
print(np.mean(scores['test_score']))

0.8554925223957948


In [10]:
# 훈련 세트를 섞은 후 10-폴드 교차 검증 수행
spliter = StratifiedKFold(n_splits=10, shuffle=True, random_state=42)
scores = cross_validate(dt, train_input, train_target, cv=spliter)
print(np.mean(scores['test_score']))

0.8581873425226026


### 하이퍼파라미터 튜닝

- 모델 파라미터 : 머신러닝 모델이 학습하는 파라미터
- 하이퍼파라미터 : 모델이 학습할 수 없어서 사용자가 지정해야만 하는 파라미터

- 머신러닝 라이브러리를 사용할 때 이런 하이퍼파라미터는 모두 클래스나 메서드의 매개변수로 표현됨

- 라이브러리가 제공하는 기본값을 그대로 사용해 모델을 훈련

- 검증 세트의 점수나 교차 검증을 통해서 매개변수 조금씩 바꿔가면서 모델을 훈련, 교차 검증 수행

- AutoML : 사람의 개입 없이 하이퍼파라미터의 튜닝을 자동으로 수행하는 기술

- max_dept와 min_sample_split 매개변수의 값이 바뀌면 함께 달라지기 때문에 이 두 매개변수를 동시에 바꿔가며 최적의 값을 찾아야 함

- 매개변수가 많아지면 문제는 더 복잡해짐

그리드 서치(grid search)
- 사이킷런에서 하이퍼파라미터 탐색과 교차 검증을 한 번에 수행하는 클래스

In [12]:
from sklearn.model_selection import GridSearchCV
# 탐색할 매개변수와 탐색할 값의 리스트를 딕셔너리로 정의
params = {'min_impurity_decrease' : [0.0001, 0.0002, 0.0003, 0.0004, 0.0005]}

In [13]:
# 탐색 대상 모델과 params 변수를 전달하여 그리드 서치 객체 생성
gs = GridSearchCV(DecisionTreeClassifier(random_state=42), params, n_jobs=-1)

In [14]:
gs.fit(train_input, train_target)

GridSearchCV(estimator=DecisionTreeClassifier(random_state=42), n_jobs=-1,
             param_grid={'min_impurity_decrease': [0.0001, 0.0002, 0.0003,
                                                   0.0004, 0.0005]})

In [15]:
dt = gs.best_estimator_
print(dt.score(train_input, train_target))

0.9615162593804117


In [16]:
print(gs.best_params_)

{'min_impurity_decrease': 0.0001}


In [18]:
print(gs.cv_results_['mean_test_score'])

[0.86800067 0.86453617 0.86492226 0.86780891 0.86761605]


In [19]:
# 앞에서 출력한 gs.best_params_와 동일한지 확인
best_index = np.argmax(gs.cv_results_['mean_test_score'])
print(gs.cv_results_['params'][best_index])

{'min_impurity_decrease': 0.0001}


1. 먼저 탐색할 매개변수 지정
2. 훈련 세트에서 그리드 서치를 수행하여 최상의 평균 검증 점수가 나오는 매개변수 조합 찾기; 이 조합은 그리드 서치 객체에 저장
3. 그리드 서치는 최상의 매개변수에서(교차 검증에 사용한 훈련 세트가 아니라) 전체 훈련 세트를 사용해 최종 모델을 훈련; 이 모델 또한 그리드 서치 객체에 저장

In [20]:
# 트리의 깊이 제한, 노드를 나누기 위한 최소 샘플 수 설정
params = {'min_impurity_decrease' : np.arange(0.0001, 0.001, 0.0001),
          'max_depth' : range(5, 20, 1),
          'min_samples_split' : range(2, 100, 10)}

- 1번 arange() 함수는 첫 번째 매개변수 값에서 시작하여 두 번째 매개변수에 도달할 때까지 세 번째 매개변수를 계속 더한 배열을 만든다

- range() 함수는 정수만 사용 가능

In [23]:
# 그리드 서치 실행
gs = GridSearchCV(DecisionTreeClassifier(random_state=42), params, n_jobs=-1)
gs.fit(train_input, train_target)

GridSearchCV(estimator=DecisionTreeClassifier(random_state=42), n_jobs=-1,
             param_grid={'max_depth': range(5, 20),
                         'min_impurity_decrease': array([0.0001, 0.0002, 0.0003, 0.0004, 0.0005, 0.0006, 0.0007, 0.0008,
       0.0009]),
                         'min_samples_split': range(2, 100, 10)})

In [24]:
# 최상의 매개변수 조합 확인
print(gs.best_params_)

{'max_depth': 14, 'min_impurity_decrease': 0.0004, 'min_samples_split': 12}


In [25]:
# 최상의 교차 검증 점수 확인
print(np.max(gs.cv_results_['mean_test_score']))

0.8683865773302731


- GridSearchCV 클래스를 사용하면 매개변수를 일일이 바꿔가며 교차 검증을 수행하지 않고 원하는 매개변수 값을 나열하면 자동으로 교차 검증을 수행해서 최상의 매개변수를 찾을 수 있음

### 랜덤 서치

랜덤 서치(random search)
- 매개변수 값의 목록을 전달하는 것이 아니라 매개변수를 샘플링할 수 있는 확률 분포 댇체를 전달

- 매개변수의 값이 수치할 때 값의 범위나 간격을 미리 정하기 어려운 경우, 너무 많은 매개변수 조건이 있어 그리드 서치 수행 시간이 오래 걸리는 경우 사용

In [26]:
from scipy.stats import uniform, randint

scipy 라이브러리 -> 적분, 보간, 대수, 확률 등으르 포함한 수치 계산 적용 라이브러리

In [28]:
# randint 객체를 만들고 10개의 숫자 샘플링
rgen = randint(0, 10)
rgen.rvs(10)

array([5, 2, 9, 9, 1, 8, 1, 7, 0, 2])

In [29]:
# 1000갸 샘플링, 각 숫자의 개수 세기
np.unique(rgen.rvs(1000), return_counts=True)

(array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9]),
 array([ 91, 112, 103, 102,  96, 111, 106,  91,  95,  93]))

In [30]:
ugen = uniform(0, 1)
ugen.rvs(10)

array([0.88913645, 0.94817059, 0.91134444, 0.3876723 , 0.28703683,
       0.39889441, 0.71378912, 0.88841267, 0.55649118, 0.01685822])

In [32]:
# 탐색할 매개변수 범위
params = {'min_impurity_decrease' : uniform(0.0001, 0.001),
            'max_depth' : randint(20, 50),
            'min_samples_split' : randint(2, 25),
            'min_samples_leaf' : randint(1, 25)}

In [33]:
# 샘플링
from sklearn.model_selection import RandomizedSearchCV
gs = RandomizedSearchCV(DecisionTreeClassifier(random_state=42), params,
                        n_iter=100, n_jobs=-1, random_state=42)
gs.fit(train_input, train_target)

RandomizedSearchCV(estimator=DecisionTreeClassifier(random_state=42),
                   n_iter=100, n_jobs=-1,
                   param_distributions={'max_depth': <scipy.stats._distn_infrastructure.rv_discrete_frozen object at 0x16aed5ad0>,
                                        'min_impurity_decrease': <scipy.stats._distn_infrastructure.rv_continuous_frozen object at 0x16aed4b50>,
                                        'min_samples_leaf': <scipy.stats._distn_infrastructure.rv_discrete_frozen object at 0x16af99b10>,
                                        'min_samples_split': <scipy.stats._distn_infrastructure.rv_discrete_frozen object at 0x16aef5d90>},
                   random_state=42)

In [34]:
# 최적의 매개변수 조합
print(gs.best_params_)

{'max_depth': 39, 'min_impurity_decrease': 0.00034102546602601173, 'min_samples_leaf': 7, 'min_samples_split': 13}


In [35]:
# 최고의 교차 검증 점수
print(np.max(gs.cv_results_['mean_test_score']))

0.8695428296438884


In [36]:
# 테스트 세트의 성능 확인
dt = gs.best_estimator_
print(dt.score(test_input, test_target))

0.86
